In [1]:
# JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [2]:
df = await fetch_data(
    index=Index.BNF,
    start_date=dt.date(2024, 1, 1),
    end_date=dt.date(2024, 1, 3),
    start_time=dt.time(9, 15),
    end_time=dt.time(15, 30),
    expiry=dt.date(2024, 1, 3),
    strike=48200,
    asset_class=AssetClass.CALL,
    ffill=True
)
print(df)

shape: (1_125, 11)
┌─────────┬───────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ datetim ┆ index ┆ expiry ┆ strike ┆ asset_ ┆ o      ┆ h      ┆ l      ┆ c      ┆ v      ┆ oi     │
│ e       ┆ ---   ┆ ---    ┆ ---    ┆ class  ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ ---     ┆ str   ┆ dateti ┆ i64    ┆ ---    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ i64    ┆ i64    │
│ datetim ┆       ┆ me[μs] ┆        ┆ str    ┆        ┆        ┆        ┆        ┆        ┆        │
│ e[μs]   ┆       ┆        ┆        ┆        ┆        ┆        ┆        ┆        ┆        ┆        │
╞═════════╪═══════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ 2024-01 ┆ bnf   ┆ 2024-0 ┆ 48200  ┆ C      ┆ 378.7  ┆ 378.7  ┆ 282.75 ┆ 283.55 ┆ 494807 ┆ 581025 │
│ -01 09: ┆       ┆ 1-03   ┆        ┆        ┆        ┆        ┆        ┆        ┆        ┆        │
│ 15:00   ┆       ┆ 00:00: ┆        ┆        ┆        ┆        ┆        